In [1]:
import spacy
import torch
import numpy
import json
import datetime
import sys
import pickle
import cupy
import pysolr
from IPython.display import Markdown, display
from numpy.testing import assert_almost_equal
nlp = spacy.load("en_pytt_bertbaseuncased_lg")

## Solr Query Utils

Used to query solr and print fancy markdown results to the notebook

In [3]:
#Solr Client
solr = pysolr.Solr('http://localhost:8983/solr/tmdb')

#Formats a search result as markdown and prints
def printmd(movieid,title,date,overview,score):
    title = '### ' + title + ' (' + str(movieid) + ')\n'
    score = '__' + str(score) + '__\n'
    date = '_' + str(date) + '_\n'
    overview = '> ' + overview
    display(Markdown(title))
    display(Markdown(score))
    display(Markdown(date))
    display(Markdown(overview))

#Print the Title, Release Date, and Overview from TMDB
def printresults(res):
    for r in res:
        printmd(r["id"],r["title"][0],r["release_date"],r["overview"][0],r["likeness"])

#Search the title and overview fields for a given query text, with a small title boost
def solrquery(q):
    return "overview:("+q+")"
        
#Enrich and Search a text query
def search(query):
    q = solrquery(query)
    print("Searching for `" + query + "` ...")
    print("  q = " + q)
    res = solr.search(q=q,fl="id,title,overview,release_date,score",rows=100)
    return res

## Rerank by tensor similarity

This method will take the top 100 results from the plain solr query, then perform a similarity operation on the query to each of those results.  The results are then reranked in order of this similarity.

In [4]:
def search_and_rerank(querytext):
    res = search(querytext)
    query = nlp(querytext).tensor
    ids = [str(r["id"]) for r in res]
    for r in res:
        movieid=str(r["id"])
        tensor=None
        likeness=0
        try:
            with open("vectors/"+movieid+".pickle", "rb") as input_file:
                 tensor = pickle.load(input_file)
        except:
            r["overview"] = [""]
            #print(movieid,"doesn't have a tensor!")
            
        if tensor is not None:
            total=0
            dims=0
            for m in tensor:
                mn=cupy.asnumpy(m)
                for mq in query:
                    total+=mn.dot(mq)
                    dims+=1
            likeness=total/dims
            #print(distance)
        r["likeness"] = likeness
    reranked = sorted(res, reverse=True, key=lambda k: k['likeness'])
    return reranked[:100]

In [9]:
res = search_and_rerank("kirk alien conspiracy")
printresults(res)

Searching for `kirk alien conspiracy` ...
  q = overview:(kirk alien conspiracy)


### The Creature Wasn't Nice (36220)


__114.78502769470215__


_1983-07-01T00:00:00Z_


> A farce satirizing extraterrestrial horror movies such as Alien.

...

> "Breaking Point" is a dramatic tale of corruption and self-realization, in which one man has to overcome a deep seeded conspiracy and his own lingering past in order to gain the redemption he desires.

### Never Make It Home (273656)


__52.22958174696914__


_2011-06-24T00:00:00Z_


> In 2006, after 10 years of constant touring with his band, Kirk Rundstrom was diagnosed with terminal cancer.  With only two months left to live, he decided to book a tour.